In [19]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader


In [20]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [21]:
train_data = datasets.ImageFolder(
    root=r"alzheimer\Combined Dataset\train",
    transform=transform
)

test_data = datasets.ImageFolder(
    root=r"alzheimer\Combined Dataset\test",
    transform=transform
)


In [22]:
train_loader = DataLoader(
    train_data, batch_size=8, shuffle=True, num_workers=0, pin_memory=True
)
test_loader = DataLoader(
    test_data, batch_size=8, num_workers=0, pin_memory=True
)

In [23]:
class Alzheimer(nn.Module):
  def __init__(self):
    super().__init__()
    self.base = models.resnet18(pretrained=True)
    self.base.fc = nn.Linear(self.base.fc.in_features,4)
  def forward(self,x):
    return self.base(x)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Alzheimer().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()

In [25]:
!pip install tqdm


In [26]:
from tqdm import tqdm
def train():
  model.train()
  total_loss = 0
  for images,labels in tqdm(train_loader, desc="Training", leave=False):
    images,labels = images.to(device),labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    loss = loss_fn(output,labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  return total_loss/len(train_loader)


In [27]:
def evaluate():
  model.eval()
  correct,total = 0,0
  with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating", leave=False):
      images,labels = images.to(device),labels.to(device)
      preds = torch.argmax(model(images),dim=1)
      correct += (preds==labels).sum().item()
      total += labels.size(0)
  return correct/total

In [28]:
for epoch in tqdm(range(1, 30), desc="Epochs"):
  avg_loss = train()
  acc = evaluate()
  print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}, Accuracy: {acc:.4f}")

Epochs:   3%|▎         | 1/29 [02:48<1:18:24, 168.01s/it]

Epoch: 1, Loss: 0.5707, Accuracy: 0.6693


Epochs:   7%|▋         | 2/29 [04:37<1:00:02, 133.43s/it]

Epoch: 2, Loss: 0.3090, Accuracy: 0.7224


Epochs:  10%|█         | 3/29 [06:30<53:46, 124.08s/it]  

...

Epoch: 27, Loss: 0.0096, Accuracy: 0.9601


Epochs:  97%|█████████▋| 28/29 [54:18<02:11, 131.69s/it]

Epoch: 28, Loss: 0.0115, Accuracy: 0.9820


Epochs: 100%|██████████| 29/29 [56:40<00:00, 117.26s/it]

Epoch: 29, Loss: 0.0032, Accuracy: 0.9750


In [29]:
torch.save(model.state_dict(), "Alzehimer.pth")

In [30]:
print("Model saved as Alzheimer.pth")

Model saved as Alzheimer.pth
